<a href="https://colab.research.google.com/github/TirendazAcademy/Deep-Learning-with-TensorFlow/blob/main/Bag-of-Words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
raw_train_ds, raw_val_ds, raw_test_ds = tfds.load(
    name="imdb_reviews",
    split=["train[:90%]","train[90%:]","test"],
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQKW7OR/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQKW7OR/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQKW7OR/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
for review, label in raw_train_ds.take(3):
  print(review.numpy().decode("utf-8"))
  print("Label: ", label.numpy() )

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
Label:  0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot developmen

In [4]:
tf.random.set_seed(42)
train_ds = raw_train_ds.shuffle(5000, seed=42).batch(32).prefetch(1)
val_ds = raw_val_ds.batch(32).prefetch(1)
test_ds = raw_test_ds.batch(32).prefetch(1)

In [5]:
text_vectorization = tf.keras.layers.TextVectorization(
    max_tokens=20000,
    output_mode="multi_hot"
)

In [6]:
text_only_train_ds = train_ds.map(lambda x, y: x)

In [7]:
text_vectorization.adapt(text_only_train_ds)

In [8]:
binary_1gram_train_ds=train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [9]:
binary_1gram_val_ds=val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_test_ds=test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [10]:
for review, label in binary_1gram_train_ds.take(1):
  print(review[0])
  print("Label: ", label[0])

tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
Label:  tf.Tensor(1, shape=(), dtype=int64)


In [11]:
def get_model(max_tokens=20000, hidden_dim=16):
  inputs= tf.keras.Input(shape=(max_tokens,))
  x = tf.keras.layers.Dense(hidden_dim, activation="relu")(inputs)
  x = tf.keras.layers.Dropout(0.5)(x)
  outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
  model =tf.keras.Model(inputs, outputs)
  model.compile(
      optimizer= "rmsprop",
      loss="binary_crossentropy",
      metrics=["accuracy"]
  )
  return model

In [12]:
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [13]:
callbacks=[tf.keras.callbacks.ModelCheckpoint(
    "binary_1gram.keras",
    save_best_only=True
)]

In [14]:
model.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds,
          epochs=10,
          callbacks=callbacks)

Epoch 1/10
704/704 [==============================] - 6s 7ms/step - loss: 0.3879 - accuracy: 0.8373 - val_loss: 0.2782 - val_accuracy: 0.8920
Epoch 2/10
704/704 [==============================] - 4s 5ms/step - loss: 0.2678 - accuracy: 0.9020 - val_loss: 0.2686 - val_accuracy: 0.8912
Epoch 3/10
704/704 [==============================] - 3s 5ms/step - loss: 0.2422 - accuracy: 0.9177 - val_loss: 0.2790 - val_accuracy: 0.8936
Epoch 4/10
704/704 [==============================] - 4s 6ms/step - loss: 0.2276 - accuracy: 0.9221 - val_loss: 0.2943 - val_accuracy: 0.8916
Epoch 5/10
704/704 [==============================] - 3s 5ms/step - loss: 0.2195 - accuracy: 0.9281 - val_loss: 0.3085 - val_accuracy: 0.8912
Epoch 6/10
704/704 [==============================] - 3s 5ms/step - loss: 0.2195 - accuracy: 0.9292 - val_loss: 0.3179 - val_accuracy: 0.8888
Epoch 7/10
704/704 [==============================] - 4s 5ms/step - loss: 0.2165 - accuracy: 0.9309 - val_loss: 0.3234 - val_accuracy: 0.8884
Epoch 

In [15]:
model = tf.keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

782/782 [==============================] - 3s 4ms/step - loss: 0.2888 - accuracy: 0.8886
Test acc: 0.889


In [16]:
text_vectorization = tf.keras.layers.TextVectorization(
    max_tokens=20000,
    output_mode="multi_hot",
    ngrams=2,
)
text_vectorization.adapt(text_only_train_ds)

In [17]:
binary_2gram_train_ds=train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_val_ds=val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_test_ds=test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [18]:
model = get_model()
callbacks=[tf.keras.callbacks.ModelCheckpoint(
    "binary_2gram.keras",
    save_best_only=True
)]
model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds,
          epochs=10,
          callbacks=callbacks)

Epoch 1/10
704/704 [==============================] - 7s 9ms/step - loss: 0.3664 - accuracy: 0.8497 - val_loss: 0.2446 - val_accuracy: 0.9016
Epoch 2/10
704/704 [==============================] - 4s 5ms/step - loss: 0.2400 - accuracy: 0.9160 - val_loss: 0.2419 - val_accuracy: 0.9016
Epoch 3/10
704/704 [==============================] - 3s 5ms/step - loss: 0.2078 - accuracy: 0.9308 - val_loss: 0.2524 - val_accuracy: 0.9056
Epoch 4/10
704/704 [==============================] - 4s 5ms/step - loss: 0.1891 - accuracy: 0.9407 - val_loss: 0.2647 - val_accuracy: 0.9056
Epoch 5/10
704/704 [==============================] - 3s 5ms/step - loss: 0.1799 - accuracy: 0.9467 - val_loss: 0.2735 - val_accuracy: 0.9056
Epoch 6/10
704/704 [==============================] - 3s 5ms/step - loss: 0.1771 - accuracy: 0.9489 - val_loss: 0.2851 - val_accuracy: 0.9024
Epoch 7/10
704/704 [==============================] - 4s 5ms/step - loss: 0.1769 - accuracy: 0.9510 - val_loss: 0.2944 - val_accuracy: 0.9044
Epoch 

In [19]:
model = tf.keras.models.load_model("binary_2gram.keras")
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

782/782 [==============================] - 3s 4ms/step - loss: 0.2709 - accuracy: 0.9020
Test acc: 0.902


In [20]:
text_vectorization = tf.keras.layers.TextVectorization(
    max_tokens=20000,
    output_mode="tf_idf",
    ngrams=2,
)
text_vectorization.adapt(text_only_train_ds)

In [21]:
tfidf_2gram_train_ds=train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_val_ds=val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_test_ds=test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [22]:
model = get_model()
callbacks=[tf.keras.callbacks.ModelCheckpoint(
    "tfidf_2gram.keras",
    save_best_only=True
)]
model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds,
          epochs=10,
          callbacks=callbacks)

Epoch 1/10
704/704 [==============================] - 9s 11ms/step - loss: 0.4647 - accuracy: 0.7923 - val_loss: 0.3842 - val_accuracy: 0.8328
Epoch 2/10
704/704 [==============================] - 4s 6ms/step - loss: 0.3177 - accuracy: 0.8697 - val_loss: 0.2782 - val_accuracy: 0.8876
Epoch 3/10
704/704 [==============================] - 5s 7ms/step - loss: 0.2975 - accuracy: 0.8749 - val_loss: 0.2632 - val_accuracy: 0.8916
Epoch 4/10
704/704 [==============================] - 4s 6ms/step - loss: 0.2806 - accuracy: 0.8900 - val_loss: 0.2696 - val_accuracy: 0.8964
Epoch 5/10
704/704 [==============================] - 5s 7ms/step - loss: 0.2603 - accuracy: 0.8973 - val_loss: 0.2792 - val_accuracy: 0.8896
Epoch 6/10
704/704 [==============================] - 4s 6ms/step - loss: 0.2558 - accuracy: 0.8984 - val_loss: 0.3028 - val_accuracy: 0.8696
Epoch 7/10
704/704 [==============================] - 4s 6ms/step - loss: 0.2488 - accuracy: 0.9004 - val_loss: 0.2980 - val_accuracy: 0.8732
Epoch

In [23]:
model = tf.keras.models.load_model("tfidf_2gram.keras")
print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

782/782 [==============================] - 6s 7ms/step - loss: 0.2922 - accuracy: 0.8826
Test acc: 0.883


In [24]:
inputs=tf.keras.Input(shape=(1,), dtype="string")
x = text_vectorization(inputs)
outputs = model(x)
inference_model = tf.keras.Model(inputs, outputs)

In [25]:
text_data= tf.convert_to_tensor(
    ["This movie is great. I liked it."])

In [26]:
predictions = inference_model(text_data)

In [27]:
print(f"{float(predictions[0]*100):.2f} percent positive")

86.51 percent positive
